In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.api as sm

import sys
sys.path.append("../")
from bargains_logit import * 

In [2]:
COST =5
WTP =  np.array([25,25])
MC = np.array([0,0])

print(simult_bargain(15,15,COST,WTP,MC,active=True))
print(seq_bargain(15, COST, WTP, MC))

#(array([15.86215466]), array([15.6463623]))
#(array([16.12045556]), array([17.17686621]))

(array([15.86215466]), array([15.6463623]))
(array([16.12045556]), array([17.17686621]))


In [3]:
def gen_data(params,num_mkts=20,nobs=100,var=5):
    wtp,cost,beta1,beta2,emc1,emc2=params
    emc = emc1,emc2
    passive_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v']) 
    active_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'])
    seq_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'])
    for j in range(0,num_mkts):
        high = 2
        mkt_wtp = np.random.uniform(high=high)
        wtp_j = np.array([wtp+mkt_wtp,wtp+mkt_wtp])
        for i in range(0,nobs):
            #sequential
            mc1 = np.random.uniform(-.3,.3,2) + np.array([emc1,emc2])
            mc2 = np.random.uniform(-.3,.3,2)
            mc = mc1+mc2

            #active results
            phi1,phi2 =simult_bargain(22,22, cost,wtp_j,mc, active=True,betas=[beta1,beta2])
            phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp_j,mc)

            row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],
                                                                                                mc1[1],wtp_j[0]]])
            active_result = pd.concat( [active_result, row] )

            #passive results
            phi1,phi2 =simult_bargain(8,8, cost,wtp_j,mc, active=False,betas=[beta1,beta2])
            phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp_j,mc)
            row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],
                                                                                                mc1[1],wtp_j[0]]])
            passive_result = pd.concat( [passive_result, row] )

            
            #seq results
            phi1,phi2 = seq_bargain(22,cost,wtp_j,mc,betas=[beta1,beta2])
            phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp_j,mc)
            row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','x1','x2','v'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],
                                                                                                mc1[1],wtp_j[0]]])
            seq_result = pd.concat( [seq_result, row] )
            #print('seq_result',[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1],wtp_j[0]]])
            
    return seq_result, active_result,passive_result


#param_list = [[25,5,.5,.5,0,0],[23,4,.5,.5,0,0],[27,7,.5,.5,0,0],[25,5,.5,.1,0,0],[25,5,.1,.5,0,0]] old list 1/25
#param_list = [[25,5,.5,.5,0,0],[20,5,.5,.5,0,0],[30,7,.5,.5,0,0],[25,5,.5,.1,0,0],[25,5,.1,.5,0,0]] old list 3/18
#param_list = [[30,5,.5,.3,0,0],[30,5,.45,.5,0,0],[30,5,.5,.5,0,0]]
param_list = [[30,5,.5,.5,0,0],[30,5,.3,.5,0,0],[30,5,.5,.1,0,0]]

for i in range(len(param_list)):
    seq_result,active_result,passive_result = gen_data(param_list[i],num_mkts=500,nobs=2,var=2)
    seq_result.to_csv('fake_data_wtp_logit_fixed2/seq_data_%s.csv'%i)
    active_result.to_csv('fake_data_wtp_logit_fixed2/active_data_%s.csv'%i)
    passive_result.to_csv('fake_data_wtp_logit_fixed2/passive_data_%s.csv'%i)

In [4]:
def run_test0(df,print_v=False):
    df['s0'] = 1 - df['s1'] - df['s2']
    res2 = sm.OLS(df['phi2'],sm.add_constant(df[['v','s0']])).fit()
    return (res2.params[1], res2.pvalues[1])

passive_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/passive_data_0.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/active_data_0.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/seq_data_0.csv',index_col=0)

print(seq_result1)

dfs = [passive_result1,active_result1,seq_result1]

for i in range(3): 
    df = dfs[i]
    df_sample = df
    if i==0:
        run_test0(df_sample,print_v=True)
    print(run_test0(df_sample))

         phi1       phi2         p1         p2        s1        s2        x1  \
0   15.886684  17.295990  24.795339  25.573407  0.380287  0.325484  0.836155   
0   16.165924  17.593475  24.842106  25.576137  0.378153  0.326521  0.525079   
0   15.504835  16.920541  24.590418  25.355142  0.383074  0.328746  0.100104   
0   16.133384  17.721819  24.628134  25.497922  0.384865  0.323415  0.002901   
0   16.215049  17.500889  25.485889  26.168603  0.379934  0.331442  0.797530   
..        ...        ...        ...        ...       ...       ...       ...   
0   15.852529  16.944484  25.374472  25.956065  0.366834  0.326553  0.931786   
0   16.139597  17.179288  25.066071  25.594186  0.378354  0.340430  0.720574   
0   16.115069  16.900066  25.318764  25.745601  0.370375  0.340069  0.415379   
0   16.127507  17.662961  24.416299  25.277484  0.376142  0.316629  0.036703   
0   15.665808  16.802145  24.545265  25.209631  0.368500  0.322649  0.483567   

          x2          v  
0   0.836155 

In [5]:
def run_test(df,print_v=False):
    df = df.copy()
    df['phi1^2'] = df['phi1']**2
    df['1/phi1'] = 1/df['phi1']
    df['1/phi1^2'] = 1/(df['phi1']**2)
    df['log(phi1)'] = np.log(df['phi1'])
    
    #clean df of weird obs
    res0= sm.OLS(df['phi2'],sm.add_constant(df[['v','phi1','phi1^2','log(phi1)','1/phi1']])).fit()
    clean_resid = np.abs(res0.resid) < 3.5*res0.resid.std()
    df_clean = df[clean_resid]
    
    #regress
    res1= sm.OLS(df['phi2'],sm.add_constant(df[['v','phi1','phi1^2','log(phi1)','1/phi1']])).fit()
    res2 = sm.OLS(df_clean['phi2'],sm.add_constant(df_clean[['v']])).fit()
    if print_v :
        print(res2.summary())
        print(res1.summary())
    return np.array([res1.pvalues[1]<=.05,res2.pvalues[1]<=.05])


passive_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/passive_data_0.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/active_data_0.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/seq_data_0.csv',index_col=0)

print(seq_result1)

dfs = [passive_result1,active_result1,seq_result1]
for i in range(3): 
    df = dfs[i]
    df_sample = df.sample(frac=0.5)
    if i==2:
        run_test(df_sample,print_v=True)
    print(run_test(df_sample))

         phi1       phi2         p1         p2        s1        s2        x1  \
0   15.886684  17.295990  24.795339  25.573407  0.380287  0.325484  0.836155   
0   16.165924  17.593475  24.842106  25.576137  0.378153  0.326521  0.525079   
0   15.504835  16.920541  24.590418  25.355142  0.383074  0.328746  0.100104   
0   16.133384  17.721819  24.628134  25.497922  0.384865  0.323415  0.002901   
0   16.215049  17.500889  25.485889  26.168603  0.379934  0.331442  0.797530   
..        ...        ...        ...        ...       ...       ...       ...   
0   15.852529  16.944484  25.374472  25.956065  0.366834  0.326553  0.931786   
0   16.139597  17.179288  25.066071  25.594186  0.378354  0.340430  0.720574   
0   16.115069  16.900066  25.318764  25.745601  0.370375  0.340069  0.415379   
0   16.127507  17.662961  24.416299  25.277484  0.376142  0.316629  0.036703   
0   15.665808  16.802145  24.545265  25.209631  0.368500  0.322649  0.483567   

          x2          v  
0   0.836155 

In [6]:
# Create an empty DataFrame to store the results
results_list = []

for j in range(5):
    # Read the dataframes
    seq_result = pd.read_csv('../GMM_est/fake_data_wtp/seq_data_%s.csv' % j, index_col=0)
    active_result = pd.read_csv('../GMM_est/fake_data_wtp/active_data_%s.csv' % j, index_col=0)
    passive_result = pd.read_csv('../GMM_est/fake_data_wtp/passive_data_%s.csv' % j, index_col=0)

    # Initialize rejection count variables
    rejections_seq = np.array([0,0])
    rejections_active = np.array([0,0])
    rejections_passive = np.array([0,0])

    nsims = 500
    
    for i in range(nsims):
        # Resample from seq dataframe
        subsample_seq = seq_result.sample(frac=0.5)
        result_seq = run_test(subsample_seq)
        rejections_seq = rejections_seq+ result_seq
        
        
        # Resample from active dataframe
        subsample_active = active_result.sample(frac=0.5)
        result_active = run_test(subsample_active)
        rejections_active = rejections_active+ result_active

        # Resample from passive dataframe
        subsample_passive = passive_result.sample(frac=0.5)
        result_passive = run_test(subsample_passive)
        rejections_passive = rejections_passive+ result_passive

    
    rejections_seq = rejections_seq/nsims
    rejections_active = rejections_active/nsims
    rejections_passive = rejections_passive/nsims
    

    
    # Add the results to the DataFrame
    results_list.append({'Round':j,'DataFrame': 'passive', 'Rejections 1': rejections_passive[0],
                         'Rejections 2': rejections_passive[1]})
    results_list.append({'Round':j,'DataFrame': 'active', 'Rejections 1': rejections_active[0],
                        'Rejections 2': rejections_active[1]})
    results_list.append({'Round':j,'DataFrame': 'seq', 'Rejections 1': rejections_seq[0],
                        'Rejections 2': rejections_seq[1]})

 
results_df = pd.DataFrame(data=results_list,columns=['Round','DataFrame','Rejections 1','Rejections 2'])

In [7]:
print(results_df[['Round','DataFrame','Rejections 2','Rejections 1']])  

    Round DataFrame  Rejections 2  Rejections 1
0       0   passive         0.000         0.050
1       0    active         1.000         1.000
2       0       seq         1.000         0.022
3       1   passive         0.032         0.104
4       1    active         1.000         1.000
5       1       seq         0.998         0.032
6       2   passive         0.144         0.336
7       2    active         1.000         1.000
8       2       seq         0.994         1.000
9       3   passive         0.002         0.020
10      3    active         1.000         1.000
11      3       seq         1.000         0.024
12      4   passive         0.002         0.308
13      4    active         1.000         1.000
14      4       seq         1.000         0.082


In [8]:
#quickly adjust for the fact that the test is a 2 stage test... only run stage 2 if i reject passive...
results_df['Round 2'] = results_df['Rejections 1'].copy()
results_df.loc[results_df['DataFrame']=='passive','Round 2'] = (1  - results_df['Rejections 2'] 
                                                             + results_df['Rejections 1'] )[results_df['DataFrame']=='passive']

print(results_df[['Round','DataFrame','Rejections 2','Round 2']])  

    Round DataFrame  Rejections 2  Round 2
0       0   passive         0.000    1.050
1       0    active         1.000    1.000
2       0       seq         1.000    0.022
3       1   passive         0.032    1.072
4       1    active         1.000    1.000
5       1       seq         0.998    0.032
6       2   passive         0.144    1.192
7       2    active         1.000    1.000
8       2       seq         0.994    1.000
9       3   passive         0.002    1.018
10      3    active         1.000    1.000
11      3       seq         1.000    0.024
12      4   passive         0.002    1.306
13      4    active         1.000    1.000
14      4       seq         1.000    0.082
